# Pseudoknot Detector

By identifying psuedoknots as a motif of interest in our RNA secondary structure dataset, we simplify the RNA secondary structure prediction problem into a *binary classification* problem.

## Preliminaries

These are things that we've already completed in previous sections, but I'll include them here for completeness so I have a better idea of what a complete ML project looks like.

1. Module imports
2. Downloader
3. FASTA file generator

In [1]:
import requests
import tarfile
import os
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split

## Comments

1. In future, BioPython SeqIO can help with file parsing (you have to use their classes which can be annoying)

In [2]:

def downloader(url, output_dir):

    # download the file, define file name based on URL
    r = requests.get(url, stream=True)
    tar_file_name = os.path.join(os.getcwd(), url.split("/")[-1])

    with open(tar_file_name, 'wb') as f:
        f.write(r.content)

    # make sure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # extract
    with tarfile.open(tar_file_name, 'r:gz') as tar:
        tar.extractall(path=output_dir)
    
    # remove the downloaded tar file after extraction
    os.remove(tar_file_name)

    print(f"Downloaded and extracted {tar_file_name} to {output_dir}")

def find_ct_files(input_dir: str) -> list:
    """
    Finds all .ct files in the given directory.
    
    :param input_dir: The directory where to look for CT files.
    :return: A list of paths to CT files.
    """
    return [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".ct")]

def extract_rna_sequence_from_ct(ct_file: str) -> tuple[str, str, int]:
    """
    Extracts the RNA sequence from a CT file. Evaluates if a pseudoknot 
    :param ct_file: Path to a CT file.
    :return: A tuple containing the filename (without extension) and the RNA sequence.
    """
    sequence = []
    with open(ct_file, 'r') as file:
        lines = file.readlines()
        # Skip the header line, process each nucleotide line
        for line in lines[1:]:
            parts = line.strip().split()
            if len(parts) < 2:
                continue
            nucleotide = parts[1]
            sequence.append(nucleotide)
    filename = os.path.splitext(os.path.basename(ct_file))[0]
    # Return the filename without extension and the sequence
    return filename, ''.join(sequence)

def save_to_fasta(input_dir, output_fasta_file):
    """
    Finds all CT files in the input directory, extracts their RNA sequences,
    and saves them in the specified FASTA file.

    :param input_dir: Directory containing CT files.
    :param output_fasta_file: Path to the output FASTA file.
    """
    ct_files = find_ct_files(input_dir)
    number = 0
    with open(output_fasta_file, 'w') as fasta_file:
        for ct_file in ct_files:
            number += 1
            filename, sequence = extract_rna_sequence_from_ct(ct_file)
            # Write the sequence in FASTA format 
            fasta_file.write(f">{filename}\n")
            fasta_file.write(f"{sequence}\n")
            fasta_file.write("\n")
    print(f"Processed {number} CT file(s)!")

archiveII = "https://rna.urmc.rochester.edu/pub/archiveII.tar.gz"
downloader(archiveII, "./gis_data")
save_to_fasta(input_dir="./gis_data/archiveII", output_fasta_file="./data.fasta")

Downloaded and extracted /Users/wwzyeo/Developer/gis/rna-ss/notebooks/archiveII.tar.gz to ./gis_data
Processed 3975 CT file(s)!


## Data Processing

Thereafter, we remove duplicate sequences, and partition the data into a train-validation-test split. We have to enable that sequences from the same cluster should be in the same data partition. Keeping RNAs from the same cluster in the same data split helps maintain independence of data splits, prevents data leakage, and results in more reliable and biologically meaningful performance evaluations. It ensures that the model learns patterns that generalise across different clusters rather than overfitting to specific groups.

4. Remove duplicate primary sequences
5. Annotate the data with an accession number and label (1: pseudoknot present, 0: pseudoknot absent)
6. Partition the data into a train-validation-test split

### Removing duplicates

In [3]:
def fasta_parse(fasta_file: str, comment='#'):
    """
    Parses a FASTA file and extracts sequence names and their corresponding sequences.

    The function reads a given FASTA file, where each sequence is identified by a line starting with 
    the '>' symbol, followed by the sequence name. The subsequent lines contain the sequence data, 
    which is concatenated into a single string for each sequence. The function also ignores any comment 
    lines that start with the specified comment character.

    Parameters:

    fasta_file : str
        The path to the FASTA file to be parsed.
    comment : str, optional
        A character indicating comment lines that should be ignored. Default is '#'.
    
    Returns:
    -------
    tuple:
        A tuple containing two lists:
        - names : List[str]
            A list of sequence names extracted from lines starting with '>'.
        - sequences : List[str]
            A list of corresponding sequences, with each sequence represented as a string.
    
    Example:
    --------
    >>> names, sequences = fasta_parse("example.fasta")
    >>> print(names)
    ['sequence1', 'sequence2']
    >>> print(sequences)
    ['ATCGATCG', 'GGCTAAGT']

    Notes:
    ------
    - Sequence names are taken from lines starting with '>', with the '>' character removed.
    - Sequences are converted to uppercase.
    - The function assumes that the sequences are stored in a standard FASTA format.

    """
    names = []
    sequences = []
    name = None
    sequence = []
    with open(fasta_file, 'r') as f:
        for line in f:
            if line.startswith(comment):
                continue
            line = line.strip()
            if line.startswith('>'):
                if name is not None:
                    names.append(name)
                    sequences.append(''.join(sequence))
                name = line[1:]
                sequence = []
            else:
                sequence.append(line.upper())
        if name is not None:
            names.append(name)
            sequences.append(''.join(sequence))

    return names, sequences

def dedup_sequences(data_tuple):
    """
    Removes duplicate sequences from a tuple containing sequence names and sequences.

    This function takes a tuple consisting of two lists: one for sequence names and 
    one for the corresponding sequences. It removes duplicate sequences and returns 
    a new tuple containing only the unique sequences and their corresponding names.

    Parameters:
    ----------
    data_tuple : tuple
        A tuple containing two lists:
        - names (list of str): A list of sequence names.
        - sequences (list of str): A list of nucleotide or protein sequences.

    Returns:
    -------
    tuple
        A tuple containing two lists:
        - result_names (list of str): A list of names corresponding to the unique sequences.
        - result_sequences (list of str): A list of unique sequences.

    Example:
    --------
    >>> names = ["seq1", "seq2", "seq3"]
    >>> sequences = ["AGCT", "CGTA", "AGCT"]
    >>> data_tuple = (names, sequences)
    >>> dedup_sequences(data_tuple)
    (["seq1", "seq2"], ["AGCT", "CGTA"])

    This function can also be modified to produce a tuple of duplicate values.
    """
    names, sequences = data_tuple
    unique_sequences = {}
    result_names = []
    result_sequences = []
    counter = 0
    
    for name, seq in zip(names, sequences):
        if seq not in unique_sequences:
            unique_sequences[seq] = name
            result_names.append(name)
            result_sequences.append(seq)
        else:
            print(f"Duplicate sequence found: {name} and {unique_sequences[seq]}")
            counter += 1

    print(f"\nTotal duplicate sequences found: {counter}")
    return (result_names, result_sequences)

def fasta_write(data_tuple, output_fasta):
    """
    Writes a tuple of names and sequences to a FASTA file.
    
    Parameters:
    ----------
    data_tuple : tuple
        A tuple where the first element is a list of names and the second element is a list of sequences.
    output_fasta : str
        The name of the output FASTA file.
    """
    names, sequences = data_tuple
    with open(output_fasta, 'w') as fasta_file:
        for name, sequence in zip(names, sequences):
            fasta_file.write(f">{name}\n{sequence}\n")


In [4]:
names, sequences = fasta_parse("data.fasta")
data_tuple = dedup_sequences((names, sequences))

# for archival, hopefully we don't have to read from here again
fasta_write(data_tuple, "dedup_data.fasta")

Duplicate sequence found: tmRNA_Yers.pest._AE009952_1-364 and tmRNA_Yers.pest._AE017042_1-364
Duplicate sequence found: srp_Leis.braz._AY781790 and srp_Leis.braz._AY722726
Duplicate sequence found: tmRNA_Heli.pylo._TRW-85962_1-386 and tmRNA_Heli.pylo._AE000511_1-386
Duplicate sequence found: 5s_Bos-taurus-1 and 5s_Mus-musculus-1
Duplicate sequence found: srp_Stap.aure._CP000253 and srp_Stap.aure._E36052
Duplicate sequence found: tRNA_tdbR00000161-Zea_mays-4577-Ile-GAU and tRNA_tdbR00000162-Spinacia_oleracea-3562-Ile-GAU
Duplicate sequence found: tRNA_tdbR00000096-Bos_taurus-9913-Phe-AA and tRNA_tdbR00000092-Oryctolagus_cuniculus-9986-Phe-AA
Duplicate sequence found: srp_Mus.musc._AC126244 and srp_Mus.musc._AC157822
Duplicate sequence found: tmRNA_Esch.coli._CP000243_1-363 and tmRNA_Esch.coli._U36840_1-363
Duplicate sequence found: srp_Yarr.lipo._M20837 and srp_Yarr.lipo._CR382127
Duplicate sequence found: RNaseP_L.japonicus-IFO12422 and RNaseP_L.japonicus-IFO15385
Duplicate sequence fo

### Labelling data

As the accession number has already been associated with each sequence from the file parsing step, now we only need to add the classification labels to our data. We can confidently detect pseudoknots with the DBN machinery we built in the first chapter.

In [5]:
def ct2dbn(ct_filename: str) -> str:
    """
    Converts a CT (Connectivity Table) file into a dot-bracket notation (DBN) string representing RNA 
    secondary structure.

    This function parses a CT file to extract the RNA sequence and base-pairing information, 
    then generates a corresponding dot-bracket notation (DBN) string. The DBN string uses 
    various types of brackets to indicate base pairs, while unpaired nucleotides are denoted by dots (`.`).

    Parameters:
    ----------
    ct_filename : str
        Path to the CT file that contains RNA sequence and base-pairing information.
        The CT file format contains nucleotide information and the indices of base-pairing partners.

    Returns:
    -------
    str
        A string in dot-bracket notation representing the RNA secondary structure.
        The string consists of dots (unpaired nucleotides) and various brackets (paired nucleotides).
        Different types of brackets (e.g., '()', '<>', '[]', '{}') represent nested and pseudoknotted structures.

    Example:
    --------
    >>> ct2dbn('example.ct')
    '..((..<<..>>..))..'

    Notes:
    ------
    - The CT file typically contains columns representing the nucleotide index, the nucleotide itself, 
      and the index of its paired nucleotide (or `0` if unpaired).
    - Pseudoknots and nested structures are represented by different levels of brackets ('()', '<>', '[]', '{}').
    - The function handles base-pairing information, deduplicates pairs, and assigns brackets at different 
      nesting levels.
    - If a pseudoknot or interleaving base pair is detected, the function advances to the next available 
      bracket level.
    - The number of bracket levels is limited by the predefined list (`levels`), which currently supports 
      up to four levels.

    """
    name = None
    sequence = []
    raw_pairlist = []

    with open(ct_filename, 'r') as f:
        for i, line in enumerate(f):
            if not i:
                name = line.split()[1]              # name of the ncRNA
            else:
                sequence.append(line.split()[1])    # save primary sequence to string

                n = int(line.split()[0])            # nucleotide index
                k = int(line.split()[4])            # base-pairing partner index
                
                # only considered paired bases (k != 0)
                if k > 0:
                    raw_pairlist.append((n, k))

    # deduplicate pairlist
    pairlist = []
    seen = set()

    for pair in raw_pairlist:
        sorted_pair = tuple(sorted(pair))
        if sorted_pair not in seen:
            seen.add(sorted_pair)
            pairlist.append(sorted_pair)

    dots = ['.'] * len(sequence)

    levels = ['()', '<>', '[]', '{}']
    current_level = 0

    for i, (start, end) in enumerate(pairlist):
        for j, (prev_start, prev_end) in enumerate(pairlist[:i]):
            # Is there any interleaving with previous pairs?
            if prev_start < start < prev_end < end:

                # Control advancement of the current_level but ensure it does not exceed the highest 
                # allowable level (determined by the length of the levels list).
                current_level = min(current_level + 1, len(levels) - 1)
                break
            else:
                current_level = 0

            # Use the current level's brackets for this base pair
        open_bracket, close_bracket = levels[current_level]
        if dots[start - 1] == "." and dots[end - 1] == ".":
            dots[start - 1] = open_bracket
            dots[end - 1] = close_bracket

    return "".join(dots)

def pseudoknot_checker(dbn: str) -> int:
    """
    Checks for the presence of pseudoknots in a dot-bracket notation (DBN) string.

    This function scans a dot-bracket notation (DBN) string to detect the presence of pseudoknots.
    A pseudoknot is typically indicated by the presence of angle brackets (`<` or `>`), which represent
    non-canonical base pairing interactions that are not nested within the usual dot-bracket structure.

    Parameters:
    ----------
    dbn : str
        A string representing RNA secondary structure in dot-bracket notation.
        Valid characters include '.', '(', ')', '[', ']', '<', '>', etc.

    Returns:
    -------
    int
        Returns 1 if pseudoknots (angle brackets '<' or '>') are detected in the DBN string, otherwise returns 0.

    Example:
    --------
    >>> dbn = "..((..<<..>>..)).."
    >>> pseudoknot_checker(dbn)
    1
    
    >>> dbn = "..((..)).."
    >>> pseudoknot_checker(dbn)
    0
    """
    pseudoknot_state = 0
    for i in dbn:
        # if "<" in dbn: 
        # even if there's a for loop in the background it's still more efficient
        # TODO: return "<" in dbn 
        if i in "<>":
            pseudoknot_state = 1
            break

    return pseudoknot_state

def add_labels_to_data(i):
    names, sequences = i
    labels = []
    for name, _ in zip(names, sequences):
        labels.append(pseudoknot_checker(ct2dbn("./gis_data/archiveII/" + name + ".ct")))
    return (names, sequences, labels)

In [6]:
labeled_data_tuple = add_labels_to_data(data_tuple)

### Clustering

In [7]:
!mmseqs easy-cluster --min-seq-id 0.9 -c 0.8 --threads 4 dedup_data.fasta data clustering_tmp

easy-cluster --min-seq-id 0.9 -c 0.8 --threads 4 dedup_data.fasta data clustering_tmp 

MMseqs Version:                     	15-6f452
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	4
k-mer length                        	0
Target search mode                  	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.8
Coverage mode                       	0
Compositional bias                  	1
Compositional bias                  	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mask residues                       	1
Mask resi

Do the clusters make sense?

In [9]:
# Load the TSV file into a DataFrame
file_path = 'data_cluster.tsv'
cluster_df = pd.read_csv(file_path, sep='\t', header=None, names=['Cluster_Rep', 'Cluster_Member'])

# Display the first few rows
cluster_df.head()

,Cluster_Rep,Cluster_Member
0,tmRNA_Dich.nodo._TRW-35819_1-352,tmRNA_Dich.nodo._TRW-35819_1-352
1,srp_Myco.hyop._AE017332,srp_Myco.hyop._AE017332
2,5s_Fusarium-cerealis-7,5s_Fusarium-cerealis-7
3,5s_Fusarium-cerealis-7,5s_Fusarium-asiaticum-6
4,5s_Fusarium-cerealis-7,5s_Fusarium-lunulosporum-4


### Data Split

Training is where the model learns optimal parameters to minimise the loss function / maximise the objective function. With the validation data, we can tune hyperparameters, which are typically aspects of the model that do not learn from training data.

In [10]:
# not enough data? 80-10-10 split
# 10 splits, 10 different validation sets (random splitting, average the performance and see what's the best)
# based on each split, different results
# TODO: k-fold cross-validation
# if you have n < 5000, n < 10000 can try this
def train_val_test_split(data_tuple, cluster_df, test_size=0.2, val_size=0.2, random_state=42):
    """
    Splits the data into train, validation, and test sets, ensuring that sequences 
    from the same cluster are only present in one of the sets.

    Parameters:
    ----------
    data_tuple : tuple
        A tuple where the first element is a list of names, the second element is a list of sequences,
        and the third element is a list of classification labels.
    cluster_df : pd.DataFrame
        A DataFrame with two columns: 'Cluster_Rep' and 'Cluster_Member', where 'Cluster_Member'
        corresponds to names in the data_tuple.
    test_size : float
        Proportion of the data to include in the test split.
    val_size : float
        Proportion of the training data to include in the validation split.
    random_state : int or None
        Random seed for reproducibility.
        
    Returns:
    -------
    train_set : tuple
        A tuple containing names, sequences, and labels for the training set.
    val_set : tuple
        A tuple containing names, sequences, and labels for the validation set.
    test_set : tuple
        A tuple containing names, sequences, and labels for the test set.
    """

    # Extract names, sequences, and labels from data_tuple
    names, sequences, labels = data_tuple

    # Map names to sequences and labels
    name_to_seq = {name: seq for name, seq in zip(names, sequences)}
    name_to_label = {name: label for name, label in zip(names, labels)}

    # Get unique clusters
    clusters = cluster_df['Cluster_Rep'].unique()

    # Split clusters into train and test sets
    train_clusters, test_clusters = train_test_split(
        clusters, test_size=test_size, random_state=random_state
    )

    # Further split the training clusters into train and validation sets
    train_clusters, val_clusters = train_test_split(
        train_clusters, test_size=val_size, random_state=random_state
    )

    # Get names for each set by filtering the cluster_df
    train_names = cluster_df[cluster_df['Cluster_Rep'].isin(train_clusters)]['Cluster_Member'].tolist()
    val_names = cluster_df[cluster_df['Cluster_Rep'].isin(val_clusters)]['Cluster_Member'].tolist()
    test_names = cluster_df[cluster_df['Cluster_Rep'].isin(test_clusters)]['Cluster_Member'].tolist()

    # Retrieve sequences and labels for the names in each set
    train_set = (
        [name for name in train_names], 
        [name_to_seq[name] for name in train_names],
        [name_to_label[name] for name in train_names]
    )
    val_set = (
        [name for name in val_names], 
        [name_to_seq[name] for name in val_names],
        [name_to_label[name] for name in val_names]
    )
    test_set = (
        [name for name in test_names], 
        [name_to_seq[name] for name in test_names],
        [name_to_label[name] for name in test_names]
    )

    return train_set, val_set, test_set



train_set, val_set, test_set = train_val_test_split(labeled_data_tuple, cluster_df, 
                                                    test_size=0.2, val_size=0.2, random_state=42)

print("Training set:",len(train_set[0]))
print("Validation set:", len(val_set[0]))
print("Test set:", len(test_set[0]))

Training set: 2215
Validation set: 560
Test set: 685


In [15]:
train_set

(['srp_Myco.hyop._AE017332',
  '5s_Fusarium-cerealis-7',
  '5s_Fusarium-asiaticum-6',
  '5s_Fusarium-lunulosporum-4',
  '5s_Fusarium-pseudograminearum-7',
  '5s_Fusarium-culmorum-6',
  'tmRNA_Myxo.xant._TRW-34_1-366',
  'srp_Bifi.long._AE014295',
  'srp_Bifi.adol._AP009256',
  'srp_Cory.effi._BA000035',
  'srp_Bifi.long._AX492782',
  '5s_Tremella-encephala-1',
  '5s_Sulfolobus-tokodaii-1',
  'tmRNA_Magn.spec._CP000471_1-366',
  '16s_H.volcanii_domain4',
  '5s_Candidatus-Portiera-aleyrodidarum-1',
  '5s_Tilletiaria-anomala-1',
  'tRNA_tdbR00000298-Asterias_amurensis-7602-Asn-GUU',
  'RNaseP_A.fulgidus',
  'RNaseP_T.acidophilum',
  'grp1_a.I1.c.S.x.townsendii.C3.tLEU',
  'grp1_a.I1.c.S.maritima.C3.tLEU',
  'RNaseP_LGB5',
  'tRNA_tdbR00000372-Triticum_aestivum-4565-Arg-CCG',
  'tmRNA_Ther.rose._TRW-309801_1-353',
  'tRNA_tdbR00000164-Saccharomyces_cerevisiae-4932-Ile-GAU',
  'tRNA_tdbR00000301-Lupinus_luteus-3873-Asn-UU',
  'RNaseP_C.hydrogenoformans',
  'tmRNA_Clav.mich._TRW-31964_1-376'

### One-Hot Encoding

In [11]:
def one_hot_encode(rna_str: str) -> np.ndarray:
    """
    Converts an RNA sequence string into a flattened one-hot encoded NumPy array.
    
    Each nucleotide in the RNA sequence is encoded as a 5-element vector:
    A -> [1, 0, 0, 0, 0] (adenine)
    C -> [0, 1, 0, 0, 0] (cytosine)
    G -> [0, 0, 1, 0, 0] (guanine)
    U -> [0, 0, 0, 1, 0] (uracil)
    X -> [0, 0, 0, 0, 1] (unknown or invalid nucleotide)

    Parameters:

    Returns:

    Example:

    """
    nucleotide_encoding = {
        'A': [1, 0, 0, 0, 0],
        'C': [0, 1, 0, 0, 0],
        'G': [0, 0, 1, 0, 0],
        'U': [0, 0, 0, 1, 0],
        'X': [0, 0, 0, 0, 1]
    }

    one_hot_encoded = []

    for nucleotide in rna_str:
        if nucleotide.upper() in nucleotide_encoding:
            one_hot_encoded.append(nucleotide_encoding[nucleotide])
        else:
            one_hot_encoded.append(nucleotide_encoding["X"])

    return np.array(one_hot_encoded).ravel()

def pad_encodings(max_length: int, encoded_sequences: list) -> np.ndarray:
    """
    Pads each sequence in a list of encoded sequences with zeros to ensure they all have the same length.

    Parameters:
    max_length (int): The length to which each sequence should be padded.
    encoded_sequences (list): A list of sequences, where each sequence is an array-like object.

    Returns:
    np.ndarray: A NumPy array containing the padded sequences.
    """

    # first we find the length of the longest sequence (in this case, we chose to do it outside)
    # max_length = max(len(seq) for seq in encoded_sequences)

 
    padded_sequences = []
    for seq in encoded_sequences:
        pad_length = max_length - len(seq)
        # then we add zeros to each sequence until it reaches that max_length
        # do that by using np.zeros() until the difference is made up
        padded_seq = np.hstack([seq, np.zeros((pad_length))]) if pad_length > 0 else seq
        padded_sequences.append(padded_seq)

    return np.array(padded_sequences)             

In [12]:
N = max(len(x) for x in test_set[1] + train_set[1] + val_set[1]) * 5

## Machine Learning

### Logistic Regression

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Step 1: Prepare the training data
# Assuming you have a function pad_encodings(N, encodings) and one_hot_encode(x)
X_train = pad_encodings(N, [one_hot_encode(seq) for seq in train_set[1]])
X_val = pad_encodings(N, [one_hot_encode(seq) for seq in val_set[1]])
X_test = pad_encodings(N, [one_hot_encode(seq) for seq in test_set[1]])

# Convert labels to numpy arrays
y_train = np.array(train_set[2])
y_val = np.array(val_set[2])
y_test = np.array(test_set[2])

# Step 2: Create and train the Logistic Regression model
model = LogisticRegression(max_iter=1000)  # Increase max_iter if the model doesn't converge

# Fit the model to the training data
model.fit(X_train, y_train)

print("LOGISTIC REGRESSION\n")

# Step 3: Evaluate the model on the validation set
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

# Classification report for validation set
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Step 4: Evaluate the model on the test set
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

# Classification report for test set
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

LOGISTIC REGRESSION

Validation Accuracy: 0.9559270516717325
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       519
           1       0.87      0.94      0.90       139

    accuracy                           0.96       658
   macro avg       0.92      0.95      0.94       658
weighted avg       0.96      0.96      0.96       658

Test Accuracy: 0.9623493975903614
Test Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       471
           1       0.95      0.92      0.93       193

    accuracy                           0.96       664
   macro avg       0.96      0.95      0.95       664
weighted avg       0.96      0.96      0.96       664



In [12]:
# \b{x}
# beta_0 + beta_1 * x_1 + beta_2 * x_2 + ... + beta_n * x_n

wT^x + b = 0 

# 15 min theory
# 45 min code review

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Step 1: Prepare the data
# Assuming you have a function pad_encodings(N, encodings) and one_hot_encode(x)
X_train = pad_encodings(N, [one_hot_encode(seq) for seq in train_set[1]])
X_val = pad_encodings(N, [one_hot_encode(seq) for seq in val_set[1]])
X_test = pad_encodings(N, [one_hot_encode(seq) for seq in test_set[1]])

# Convert labels to numpy arrays
y_train = np.array(train_set[2])
y_val = np.array(val_set[2])
y_test = np.array(test_set[2])

# Step 2: Set up parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],          # Regularization strength
    'penalty': ['l1', 'l2'],                      # L1 and L2 regularization
    'solver': ['liblinear'],                      # 'liblinear' works with both 'l1' and 'l2' penalties
    'max_iter': [100, 200, 500, 1000]             # Maximum iterations for solver convergence
}

# Step 3: Initialize Logistic Regression model
log_reg = LogisticRegression()

# Step 4: Use GridSearchCV to tune hyperparameters
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print("Best Parameters from GridSearchCV:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Step 5: Train the best model using the training data
best_model = grid_search.best_estimator_

# Step 6: Evaluate the model on the validation set
y_val_pred = best_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

# Validation Classification Report
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Step 7: Evaluate the model on the test set
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

# Test Classification Report
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

Best Parameters from GridSearchCV: {'C': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Best Cross-Validation Accuracy: 0.9564917157302633
Validation Accuracy: 0.952887537993921
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       519
           1       0.86      0.94      0.89       139

    accuracy                           0.95       658
   macro avg       0.92      0.95      0.93       658
weighted avg       0.96      0.95      0.95       658

Test Accuracy: 0.9623493975903614
Test Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       471
           1       0.94      0.93      0.93       193

    accuracy                           0.96       664
   macro avg       0.96      0.95      0.95       664
weighted avg       0.96      0.96      0.96       664



### Random Forest Classifier

A random forest is an ensemble of decision trees.

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Step 1: Prepare the training data
# Assuming you have a function pad_encodings(N, encodings) and one_hot_encode(x)
X_train = pad_encodings(N, [one_hot_encode(seq) for seq in train_set[1]])
X_val = pad_encodings(N, [one_hot_encode(seq) for seq in val_set[1]])
X_test = pad_encodings(N, [one_hot_encode(seq) for seq in test_set[1]])

# Convert labels to numpy arrays
y_train = np.array(train_set[2])
y_val = np.array(val_set[2])
y_test = np.array(test_set[2])

# Step 2: Create and train the Logistic Regression model
model = RandomForestClassifier(n_estimators=100)  # Increase n_estimators if the model doesn't converge

# Fit the model to the training data
model.fit(X_train, y_train)

print("RANDOM FOREST CLASSIFIER\n")

# Step 3: Evaluate the model on the validation set
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

# Classification report for validation set
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Step 4: Evaluate the model on the test set
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

# Classification report for test set
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

RANDOM FOREST CLASSIFIER

Validation Accuracy: 0.9407294832826748
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       519
           1       0.81      0.94      0.87       139

    accuracy                           0.94       658
   macro avg       0.90      0.94      0.92       658
weighted avg       0.95      0.94      0.94       658

Test Accuracy: 0.9412650602409639
Test Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.94      0.96       471
           1       0.87      0.93      0.90       193

    accuracy                           0.94       664
   macro avg       0.92      0.94      0.93       664
weighted avg       0.94      0.94      0.94       664



In [13]:
# The same thing as above, but with hyperparameter tuning

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Step 1: Prepare the data
# Assuming you have a function pad_encodings(N, encodings) and one_hot_encode(x)
X_train = pad_encodings(N, [one_hot_encode(seq) for seq in train_set[1]])
X_val = pad_encodings(N, [one_hot_encode(seq) for seq in val_set[1]])
X_test = pad_encodings(N, [one_hot_encode(seq) for seq in test_set[1]])

# Convert labels to numpy arrays
y_train = np.array(train_set[2])
y_val = np.array(val_set[2])
y_test = np.array(test_set[2])

# Step 2: Set up parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 500],        # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],        # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],        # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],          # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False]              # Whether bootstrap samples are used when building trees
}

# Step 3: Initialize RandomForestClassifier
rf_model = RandomForestClassifier(n_jobs=-1)

# Step 4: Use GridSearchCV to tune hyperparameters
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print("Best Parameters from GridSearchCV:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Step 5: Train the best model using the training data
best_model = grid_search.best_estimator_

# Step 6: Evaluate the model on the validation set
y_val_pred = best_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

# Validation Classification Report
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Step 7: Evaluate the model on the test set
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

# Test Classification Report
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

Best Parameters from GridSearchCV: {'bootstrap': False, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Cross-Validation Accuracy: 0.94627539503386
Validation Accuracy: 0.9446428571428571
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       413
           1       0.85      0.95      0.90       147

    accuracy                           0.94       560
   macro avg       0.92      0.95      0.93       560
weighted avg       0.95      0.94      0.95       560

Test Accuracy: 0.9503649635036496
Test Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       494
           1       0.89      0.94      0.91       191

    accuracy                           0.95       685
   macro avg       0.93      0.95      0.94       685
weighted avg       0.95      0.95      0.95       685



In [14]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Step 1: Prepare the data
# Assuming you have a function pad_encodings(N, encodings) and one_hot_encode(x)
X_train = pad_encodings(N, [one_hot_encode(seq) for seq in train_set[1]])
X_val = pad_encodings(N, [one_hot_encode(seq) for seq in val_set[1]])
X_test = pad_encodings(N, [one_hot_encode(seq) for seq in test_set[1]])

# Convert labels to numpy arrays
y_train = np.array(train_set[2])
y_val = np.array(val_set[2])
y_test = np.array(test_set[2])

# Step 2: Set up parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 500],       # Number of boosting rounds
    'max_depth': [3, 6, 10],               # Maximum depth of a tree
    'learning_rate': [0.001, 0.01, 0.1],   # Step size shrinkage to prevent overfitting
    'subsample': [0.6, 0.8, 1.0],          # Fraction of samples used for training each tree
    'colsample_bytree': [0.6, 0.8, 1.0],   # Fraction of features used for training each tree
    'gamma': [0, 0.1, 0.2],                # Minimum loss reduction to make a split
    'reg_lambda': [1, 10, 100]             # L2 regularization term on weights
}

# Step 3: Initialize XGBClassifier
xgb_model = XGBClassifier(eval_metric='logloss', n_jobs=-1)

# Step 4: Use GridSearchCV to tune hyperparameters
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print("Best Parameters from GridSearchCV:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Step 5: Train the best model using the training data
best_model = grid_search.best_estimator_

# Step 6: Evaluate the model on the validation set
y_val_pred = best_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

# Validation Classification Report
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Step 7: Evaluate the model on the test set
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

# Test Classification Report
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

Best Parameters from GridSearchCV: {'colsample_bytree': 1.0, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 500, 'reg_lambda': 100, 'subsample': 0.8}
Best Cross-Validation Accuracy: 0.9589164785553047
Validation Accuracy: 0.9678571428571429
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       413
           1       0.93      0.95      0.94       147

    accuracy                           0.97       560
   macro avg       0.96      0.96      0.96       560
weighted avg       0.97      0.97      0.97       560

Test Accuracy: 0.9576642335766423
Test Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       494
           1       0.93      0.92      0.92       191

    accuracy                           0.96       685
   macro avg       0.95      0.95      0.95       685
weighted avg       0.96      0.96      0.9

Broad order: hyperparameters, how many estimators? how deep should the tree be? how many features should be used? how many samples should be used? how many boosting rounds? how much should the learning rate be? how much should the subsample be? how much should the colsample_bytree be? how much should the gamma be? how much should the reg_lambda be?


don't need to go through the math step by step

theory for both
results for both

learning is an iterative process